In [ ]:
pip install influx_line_protocol
pip install pycoingecko

In [ ]:
import requests
import pandas as pd
from pycoingecko import CoinGeckoAPI

cg = CoinGeckoAPI()

## Create table

In [ ]:
q = 'create table crypto_prices'\
    '(timestamp timestamp,'\
    'BTC double,'\
    'ETH double,'\
    'LUNA double)'

r = requests.get("http://localhost:9000/exec?query=" + q)
print(r.status_code)

## Pull Data and insert into table

In [ ]:
prices = cg.get_price(ids=['bitcoin', 'ethereum','terra-luna'], vs_currencies='usd')
prices

In [ ]:
import psycopg2

try:
    connection = psycopg2.connect(user="admin",
                                  password="quest",
                                  host="127.0.0.1",
                                  port="8812",
                                  database="qdb")

    cursor = connection.cursor()
    print('Connection established.')

    for i in range(30):
        # Obtain price data
        prices = cg.get_price(ids=['bitcoin', 'ethereum','terra-luna'], vs_currencies='usd')

        # Insert price data
        cursor.execute(f"""
            INSERT INTO crypto_prices
            VALUES (
                systimestamp(),
                {float(prices['bitcoin']['usd'])},
                {float(prices['ethereum']['usd'])},
                {float(prices['terra-luna']['usd'])}
                );
            """)

        sleep(3)

    # Commit records
    connection.commit()

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    # Closing database connection.
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

## Query Data

In [ ]:
import io

r = requests.get("http://localhost:9000/exp?query=select * from  crypto_prices")
df = pd.read_csv(io.StringIO(r.text), parse_dates=['timestamp'], index_col='timestamp')

df.head()

In [ ]:
df['ETH'].plot()

## Delete Table

In [ ]:
r = requests.get("http://localhost:9000/exec?query=drop table crypto_prices")